In [26]:
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.retrievers import BM25Retriever
from langchain_classic.retrievers import EnsembleRetriever
from langchain_core.documents import Document
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
# chain creation
from langchain_classic.chains import create_retrieval_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain_classic.chains.combine_documents import create_stuff_documents_chain
from langchain_core.output_parsers import StrOutputParser 
from langchain.chat_models import init_chat_model
from langchain_groq import ChatGroq
from dotenv import load_dotenv

In [3]:
loader = TextLoader("تال چیست.txt",encoding="utf-8")
loader = loader.load()


In [20]:
splitter = RecursiveCharacterTextSplitter(
    separators=[" "],
    chunk_size=600,
    chunk_overlap=50
)
docs = splitter.split_documents(loader)
for d in docs[:-3]:
    print("*"*50)
    print(d)

**************************************************
page_content='از ۲۰ آبان ۱۴۰۴، مرحله‌ای جدید با عنوان معاملات پایانی (TAL) به فرآیند معاملات بورس ایران اضافه شده است.
در این مرحله، معامله‌گران می‌توانند از ساعت ۱۲:۴۵ تا ۱۳:۰۰ نمادهای مشخصی را با قیمت پایانی ساعت ۱۲:۳۰ معامله کنند.

هدف از اجرای این مرحله، بهبود کیفیت کشف قیمت، کاهش هیجانات لحظه‌ای دقایق پایانی بازار و فراهم‌کردن امکان تعدیل روزانه‌ی پرتفوی سرمایه‌گذاران است.

اهمیت مرحله TAL در معاملات بورس
در معاملات عادی، قیمت‌ها ممکن است تحت تأثیر سفارش‌های هیجانی در دقایق پایانی نوسان شدیدی داشته باشند.
مرحله TAL با ایجاد امکان معامله بر اساس قیمت پایانی، به بازار نظم بیشتری می‌دهد.

مزایای اصلی این' metadata={'source': 'تال چیست.txt'}
**************************************************
page_content='به بازار نظم بیشتری می‌دهد.

مزایای اصلی این مرحله عبارت‌اند از:

🔸انجام معاملات با قیمت ثابت و قابل پیش‌بینی؛ کاهش نوسانات هیجانی.

🔸افزایش شفافیت و انضباط معاملاتی در پایان روز.

🔸امکان تحلیل رفتار بازار برای روز بعد بر اساس داده‌ه

In [21]:
bm25 = BM25Retriever.from_documents(documents=docs)
bm25.k = 2

embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")

dense_vectorstore = FAISS.from_documents(docs, embedding_model)

dense_retriever = dense_vectorstore.as_retriever()

sparse_retriever=BM25Retriever.from_documents(docs)
sparse_retriever.k=2 # top k documents to retriever

# Combine with Ensemble Retriever
hybrid_retriever=EnsembleRetriever(
    retrievers=[dense_retriever,sparse_retriever],
    weights=[0.7,0.3]
)

In [34]:
docstest = hybrid_retriever.invoke("نکات ثبت سفارش")
# for d in docstest[:-3]:
#     print("*"*50)
#     print(d)
doc_lines = [f"{i+1}. {doc.page_content}" for i, doc in enumerate(docstest)]
formatted_docs = "\n".join(doc_lines)

In [36]:
doc_lines

['1. و فروش فقط با قیمت پایانی سهم در ساعت ۱۲:۳۰ انجام می\u200cشود.\n\nمرحله\n\nزمان\n\nتوضیح\n\nپیش\u200cگشایش TAL\n\n۱۲:۴۵ تا ۱۲:۵۰\n\nامکان ثبت، حذف یا ویرایش سفارش\u200cها روی قیمت پایانی\n\nشروع معاملات\n\n۱۲:۵۰\n\nتطبیق سفارش\u200cها و آغاز معاملات روی قیمت پایانی\n\nادامه معاملات\n\n۱۲:۵۰ تا ۱۳:۰۰\n\nمعاملات به\u200cصورت پیوسته روی قیمت پایانی ادامه دارد\n\n\n\n\nنکات مهم ثبت سفارش در TAL\n🔸سفارش\u200cها قابل ثبت، حذف و ویرایش هستند.\n\n🔸اعتبار سفارش باید Day (روزانه) باشد.\n\n🔸تنها نمادهایی که بورس اعلام کرده، مشمول TAL هستند.\n\n🔸نمادهای فعال در این مرحله با عدد ۴ در انتهای نام نمایش داده می\u200cشوند (مثلاً: فولاد۴).\n\n🔸برای معامله در TAL، امضای بیانیه ریسک',
 '2. دقیق\u200cتری در پایان روز بگیرند و از فرصت\u200cهای معاملاتی این بازه زمانی استفاده کنند.',
 '3. از ۲۰ آبان ۱۴۰۴، مرحله\u200cای جدید با عنوان معاملات پایانی (TAL) به فرآیند معاملات بورس ایران اضافه شده است.\nدر این مرحله، معامله\u200cگران می\u200cتوانند از ساعت ۱۲:۴۵ تا ۱۳:۰۰ نمادهای مشخصی را با قیمت پایانی ساعت ۱

In [91]:
system_prompt = """"تو یک دستیار مفید هستی که باید اطلاعات دریافتی را بر اساس ارتباط آن با سوال کاربر شماره گذاری و از بیشترین به کمترین ارتباط  مرتب کنی 
output format : [2,3,4,1]
اطلاعات : {context}
سوال کاربر : {question}
"""
template = ChatPromptTemplate.from_messages(
            [
                ("system", system_prompt),
                ("human", "{question}"),
            ]
        )
template

ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template='"تو یک دستیار مفید هستی که باید اطلاعات دریافتی را بر اساس ارتباط آن با سوال کاربر شماره گذاری و از بیشترین به کمترین ارتباط  مرتب کنی \noutput format : [2,3,4,1]\nاطلاعات : {context}\nسوال کاربر : {question}\n'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, template='{question}'), additional_kwargs={})])

In [108]:
import os
load_dotenv()
os.environ["GROQ_API"]=os.getenv("GROQ_API")
groq_llm = ChatGroq(model="llama-3.1-8b-instant",api_key=os.getenv("GROQ_API"))

In [109]:
chain = template | groq_llm | StrOutputParser()
chain

ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template='"تو یک دستیار مفید هستی که باید اطلاعات دریافتی را بر اساس ارتباط آن با سوال کاربر شماره گذاری و از بیشترین به کمترین ارتباط  مرتب کنی \noutput format : [2,3,4,1]\nاطلاعات : {context}\nسوال کاربر : {question}\n'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, template='{question}'), additional_kwargs={})])
| ChatGroq(profile={'max_input_tokens': 131072, 'max_output_tokens': 8192, 'image_inputs': False, 'audio_inputs': False, 'video_inputs': False, 'image_outputs': False, 'audio_outputs': False, 'video_outputs': False, 'reasoning_output': False, 'tool_calling': True}, client=<groq.resources.chat.completions.Completions object at 0x0000012359AD5350>, as

In [111]:
response = chain.invoke({"context":doc_lines,"question":"نکات ثبت سفارش"}) # vpn connection error by retesting
print(response)

[4, 1, 3, 2]


In [112]:
type(response)
response[:10]

'[4, 1, 3, '

In [113]:
indices = [int(x.strip()) - 1 for x in response.split(",") if x.strip().isdigit()] # it fails bcz first and last one is not digit
indices

[0, 2]

In [114]:
# Regex extract numbers from brackets
import re

match = re.search(r"\[(.*?)\]", response)
if not match:
    raise ValueError("No ranking list found")

indices = [int(x.strip()) - 1 for x in match.group(1).split(",")]
indices


[3, 0, 2, 1]

In [115]:
doc_lines

['1. و فروش فقط با قیمت پایانی سهم در ساعت ۱۲:۳۰ انجام می\u200cشود.\n\nمرحله\n\nزمان\n\nتوضیح\n\nپیش\u200cگشایش TAL\n\n۱۲:۴۵ تا ۱۲:۵۰\n\nامکان ثبت، حذف یا ویرایش سفارش\u200cها روی قیمت پایانی\n\nشروع معاملات\n\n۱۲:۵۰\n\nتطبیق سفارش\u200cها و آغاز معاملات روی قیمت پایانی\n\nادامه معاملات\n\n۱۲:۵۰ تا ۱۳:۰۰\n\nمعاملات به\u200cصورت پیوسته روی قیمت پایانی ادامه دارد\n\n\n\n\nنکات مهم ثبت سفارش در TAL\n🔸سفارش\u200cها قابل ثبت، حذف و ویرایش هستند.\n\n🔸اعتبار سفارش باید Day (روزانه) باشد.\n\n🔸تنها نمادهایی که بورس اعلام کرده، مشمول TAL هستند.\n\n🔸نمادهای فعال در این مرحله با عدد ۴ در انتهای نام نمایش داده می\u200cشوند (مثلاً: فولاد۴).\n\n🔸برای معامله در TAL، امضای بیانیه ریسک',
 '2. دقیق\u200cتری در پایان روز بگیرند و از فرصت\u200cهای معاملاتی این بازه زمانی استفاده کنند.',
 '3. از ۲۰ آبان ۱۴۰۴، مرحله\u200cای جدید با عنوان معاملات پایانی (TAL) به فرآیند معاملات بورس ایران اضافه شده است.\nدر این مرحله، معامله\u200cگران می\u200cتوانند از ساعت ۱۲:۴۵ تا ۱۳:۰۰ نمادهای مشخصی را با قیمت پایانی ساعت ۱

In [116]:
indices

[3, 0, 2, 1]

In [117]:
reranked_docs = [doc_lines[num] for i, num in enumerate (indices)]
reranked_docs

['4. به بازار نظم بیشتری می\u200cدهد.\n\nمزایای اصلی این مرحله عبارت\u200cاند از:\n\n🔸انجام معاملات با قیمت ثابت و قابل پیش\u200cبینی؛ کاهش نوسانات هیجانی.\n\n🔸افزایش شفافیت و انضباط معاملاتی در پایان روز.\n\n🔸امکان تحلیل رفتار بازار برای روز بعد بر اساس داده\u200cهای پایانی.\n\n🔸کمک به مدیریت بهتر پرتفوی در انتهای بازار.\n\nبه بیان ساده، TAL ابزاری است برای کشف دقیق\u200cتر قیمت و کنترل هیجان انتهای بازار. با این شرایط معامله\u200cگران حرفه\u200cای می\u200cتوانند با تحلیل رفتار معاملات TAL، روند احتمالی روز بعد را بهتر پیش\u200cبینی کنند.\n\nنحوه انجام معاملات TAL\nدر مرحله TAL، خرید و فروش فقط با قیمت پایانی سهم در ساعت ۱۲:۳۰ انجام',
 '1. و فروش فقط با قیمت پایانی سهم در ساعت ۱۲:۳۰ انجام می\u200cشود.\n\nمرحله\n\nزمان\n\nتوضیح\n\nپیش\u200cگشایش TAL\n\n۱۲:۴۵ تا ۱۲:۵۰\n\nامکان ثبت، حذف یا ویرایش سفارش\u200cها روی قیمت پایانی\n\nشروع معاملات\n\n۱۲:۵۰\n\nتطبیق سفارش\u200cها و آغاز معاملات روی قیمت پایانی\n\nادامه معاملات\n\n۱۲:۵۰ تا ۱۳:۰۰\n\nمعاملات به\u200cصورت پیوسته روی قیمت پایانی ادا

### put them all in one chain
- we cant do it with one only question input bcz ai reranker needs normalizer pipeline

In [121]:
system_prompt = """
تو یک دستیار فارسی‌زبان مفید هستی.
از اطلاعات داده‌شده برای پاسخ دقیق به سوال استفاده کن.
پاسخ باید روان، طبیعی، و مناسب خواندن توسط انسان باشد.
از استفاده از هر زبان یا کاراکتر غیر فارسی (مانند چینی، انگلیسی یا عربی) خودداری کن.


اطلاعات: {context}
سوال کاربر : {question}
"""
answer_template = ChatPromptTemplate.from_messages(
            [
                ("system", system_prompt),
                ("human", "{question}"),
            ]
        )
answer_template

ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template='\nتو یک دستیار فارسی\u200cزبان مفید هستی.\nاز اطلاعات داده\u200cشده برای پاسخ دقیق به سوال استفاده کن.\nپاسخ باید روان، طبیعی، و مناسب خواندن توسط انسان باشد.\nاز استفاده از هر زبان یا کاراکتر غیر فارسی (مانند چینی، انگلیسی یا عربی) خودداری کن.\n\n\nاطلاعات: {context}\nسوال کاربر : {question}\n'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, template='{question}'), additional_kwargs={})])

In [ ]:
answer_chain = create_stuff_documents_chain(groq_llm,answer_template)
full_chain = create_retrieval_chain

In [125]:
# put them all in one full_chain
from langchain_core.runnables import RunnableMap
full_chain = (
    RunnableMap(
        question = lambda x: x['question'],
        context = reranked_docs)
    |groq_llm
    |answer_template
)
full_chain

TypeError: Expected a Runnable, callable or dict.Instead got an unsupported type: <class 'list'>

In [124]:
print(full_chain.invoke({'question':"نکات ثبت سفارش"}))

TypeError: unhashable type: 'list'